In [1]:
import feature_engineering as fe
import train_utils as tu

In [ ]:
x, xx = fe.oh_med()

In [ ]:
alldata  =  fe.pd.read_csv("train.csv")
y = alldata.Response

In [ ]:
corrs = []
for i in range(x.shape[1]):
    corrs.append(fe.np.corrcoef(x[:, i], y)[0][1])

In [ ]:
good_corrs = filter(lambda (_, x): abs(x) > 0.01, enumerate(corrs))
good_inds = [i for i, _ in good_corrs]
len(good_inds)

In [ ]:
x[:, good_inds].shape, xx[:, good_inds].shape

In [ ]:
nx, nxx = fe.cut_uncorrelated(x, xx, y, 0.01)

In [ ]:
def oh_med_cut():
    x, xx = fe.oh_med()
    y = fe.get_y()
    return fe.cut_uncorrelated(x, xx, y, 0.01)

In [ ]:
x, xx = oh_med_cut()

In [ ]:
x.shape, xx.shape

In [ ]:
x.shape, xx.shape

In [ ]:
nx.shape

In [ ]:
x, xx = fe.fe2()

In [ ]:
tu.make_sub_optimized(tu.linreg(), sorted([tu.linreg(), tu.xgbr()]), "fe2_km20", "linlinxgbrfe2km20.csv")

INFO:trainlo:optimising offsets 59381
INFO:trainlo:done optimising offsets 59381
